# Decision Trees: Programming Practice

COSC 410: Applied Machine Learning\
Colgate University\
*Prof. Apthorpe*

## Overview

This notebook will give you practice with the following topics:
  1. Training decision tree models for classification
  2. Visualizing decision trees and decision tree performance
  3. Using decision trees to measure feature importance
  
We will be using a combination of available datasets from the American Community Survey (ACS) and the Federal Communications Commission (FCC).  The goal will be to understand the nature of broadband deployment across the city of Chicago, specifically what geographic and demographic features turn out to be good predictors of broadband deployment. This question is particularly relevant in the new "Zoom era," as access to high-speed Internet connections can determine access to education, jobs, and social support. 

The existence of broadband connectivity in a certain area can be measured in a variety of ways (e.g., available speed tiers, subscriptions, measured performance) and reported in different ways (e.g., by ISPs or by citizens/subscribers). We will use data reported by ISPs, who are required to report their service offerings by Census block to the FCC yearly. 

**NOTE:** You will need run the following pip install commands, and then restart jupyter-lab for this notebook to work: 
* `pip install sodapy`
* `pip install geopandas`
* `pip install descartes`
* `pip install censusdata`
* `pip install pydotplus`

You MAY also need to install [Graphviz](https://graphviz.org/), but you should try the notebook without it first.

This notebook is adapted from the University of Chicago's Machine Learning for Public Policy course.


## Part 1. Data Import & Exploration

#### FCC Broadband Map Data

Given that the broadband data reported to the FCC is a large dataset, we will work with a truncated version specifically for Chicago. The FCC makes this data available for exploration on its [website]!(https://broadbandmap.fcc.gov/) and [for download](https://broadbandmap.fcc.gov/#/data-download). The specific data that we will use today is from June 2019 in Cook County, IL. The following code downloads the data according to the API documentation. The download should take no more than 5 minutes.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from sodapy import Socrata

client = Socrata("opendata.fcc.gov", None)

# Census blockcode is the only geographic attribute, so we limit the data to Cook County 
# Returned as JSON from API and converted to Python list of dictionaries by sodapy 
results = client.get("c67e-jknh", limit=800000, where="starts_with(blockcode, '17031')")

# Convert to pandas DataFrame
fcc_df = pd.DataFrame.from_records(results)

# Sanity check shape and first 2 rows
print(fcc_df.shape)
fcc_df.head(2)

We should explore this data with visualizations before diving into the machine learning. Specifically, we will produce the following three maps for the City of Chicago:
1. The maximum contractual downstream speed offered by any provider in each Census block group.
2. The number of unique ISPs that offer service in each Census block group.
3. The number of unique ISPs that offer service at or above 25 Mbps downstream and 3 Mbps upstream in each Census block group. (This is the FCC's definition of broadband Internet access).

We will do this by joining the Census Block code in each row of our processed FCC data with the City of Chicago's geographic data. 

In [ ]:
# Recast up/down speed data from text to numeric
fcc_df['maxcirdown'] = fcc_df['maxcirdown'].astype(float)
fcc_df['maxcirup'] = fcc_df['maxcirup'].astype(float)

# Get 12-digit block group FIPS code
fcc_df['block_group_id'] = fcc_df['blockcode'].str.slice(0, 12)

# Aggregate data to answer questions
max_speed_by_blockgroup = fcc_df.groupby('block_group_id').agg({'maxcirdown': 'max'}).rename(columns={'maxcirdown': 'max_speed'})
num_isp_by_blockgroup = fcc_df.groupby('block_group_id').agg({'provider_id': pd.Series.nunique}).rename(columns={'provider_id': 'num_isp'})
num_broadband_by_blockgroup = fcc_df[(fcc_df['maxcirdown'] >= 25) & (fcc_df['maxcirup'] >= 3)].groupby('block_group_id').agg({'provider_id': pd.Series.nunique}).rename(columns={'provider_id': 'num_broadband'})

# Join aggregated data into one dataframe
data_by_blockgroup = max_speed_by_blockgroup.join([num_isp_by_blockgroup, num_broadband_by_blockgroup])

# Replace NaNs with 0 and convert counts to ints
data_by_blockgroup = data_by_blockgroup.fillna(value={'num_broadband': 0}).reset_index()
data_by_blockgroup['num_broadband'] = data_by_blockgroup['num_broadband'].astype(int)

# Load census block boundaries geojson
client = Socrata("data.cityofchicago.org", None)
results = client.get("bt9m-d2mf", content_type="geojson", limit=50000)

# Convert to pandas DataFrame and aggregate maps at the block group level
blocks = gpd.GeoDataFrame.from_features(results)
blocks['block_group_id'] = blocks['geoid10'].str.slice(0, 12)
blockgroups = blocks.dissolve(by='block_group_id')

# Join onto boundary df
fcc_map = blockgroups.merge(data_by_blockgroup, how='left', on='block_group_id').set_index('block_group_id')

print(fcc_map.shape)
fcc_map.head(2)

In [ ]:
# PLOT: Maximum advertised downstream speed offered by any provider 
map1 = fcc_map.plot(column='max_speed', cmap="Blues", edgecolor="0.5", linewidth= 0.1, figsize=(10, 7), legend=True)
map1.set_title("Maximum contractually obligated downstream speed by block")

In [ ]:
# PLOT: The number of unique ISPs that offer service 
map2 = fcc_map.plot(column='num_isp', cmap="Blues", edgecolor="0.5", linewidth= 0.1, figsize=(10, 7), legend=True)
map2.set_title("Number of unique ISPs")

In [ ]:
# PLOT: Number of unique ISPs that offer service 25+ Mbps downstream and 3+ Mbps upstream 
map3 = fcc_map.plot(column='num_broadband', cmap="Blues", edgecolor="0.5", linewidth= 0.1, figsize=(10, 7), legend=True)
map3.set_title("Number of unique ISPs with broadband service")

#### ACS Data

The American Community Survey (ACS) provides data on broadband Internet access subscriptions, as reported by participants. For this lab, we will use the ACS 5-year estimates at the Census block group level for 2018.

The following cell uses the Census API to perform the following for block groups in the City of Chicago:

1. Load ACS data for percentages of broadband Internet access of any type
2. Load ACS data for Total Population, number of white residents, number of Black residents, and median income – and then compute the following:
   * the percentage of each Census block's population that is white and Black
   * the median income for the block
   * the population density of the block (e.g. in units of population per square kilometer)

In [ ]:
import censusdata

# Pull ACS data 
census_tables = {
    'GEO_ID': 'GEO_ID', 
    'B28011_001E': 'Internet Total', 
    'B28011_004E': 'Broadband', 
    'B02001_001E': 'Race Total', 
    'B02001_002E': 'White', 
    'B02001_003E': 'Black', 
    'B19013_001E': 'Median Income'}
acs_df = censusdata.download("acs5", 2018, censusdata.censusgeo([("state", "17"), ("county", "031"), ("tract", "*"), ("block group", "*")]), list(census_tables.keys()))
acs_df.rename(columns=census_tables, inplace=True)

# Create percentage variables 
acs_df['pct_broadband_customer_reported'] = acs_df['Broadband'] / acs_df['Internet Total']
acs_df['pct_white'] = acs_df['White'] / acs_df['Race Total']
acs_df['pct_black'] = acs_df['Black'] / acs_df['Race Total']

# Population density is not directly available in ACS - must calculate manually
# Units are in people per square kilometer
blockgroups.crs = "EPSG:4326"
blockgroups['area'] = blockgroups.to_crs({'init': 'epsg:3857'}).area / (10**6)
acs_df['block_group_id'] = acs_df['GEO_ID'].str.slice(9, 21)
acs_df = acs_df.set_index('block_group_id').join(blockgroups['area'], how='right')
acs_df['pop_density'] = acs_df['Race Total'] / acs_df['area']

acs_df.head(2)

## Part 2: Decision Tree Prediction

We will next attempt to train a decision tree prediction model to predict ISP-reported broadband Internet access from ACS data (at the Census block level).

In [ ]:
# Setup
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

First we will merge the ACS data with the FCC data and split into train and test sets

In [ ]:
# Merge census and broadband data 
merged_df = acs_df[['Median Income', 'pct_broadband_customer_reported', 'pct_white', 'pct_black', 'pop_density']].join(fcc_map[['max_speed', 'num_isp', 'num_broadband']], how='right').rename(columns={'Median Income': 'median_income'})

In [ ]:
# Divide data into 80%/20% train/test sets
train_df, test_df = train_test_split(merged_df, test_size = 0.2, random_state=0)

Next, we will clean-up missing or invalid values

In [ ]:
# Clean missing income values 
train_df["median_income"].replace({-666666666.0: np.NAN}, inplace=True)
test_df["median_income"].replace({-666666666.0: np.NAN}, inplace=True)

In [ ]:
# Impute NAs with column means 
train_means = train_df.mean()
train_df = train_df.fillna(train_means)
test_df = test_df.fillna(train_means)

Then we separate out the labels from the features, converting the labels into binary (has broadband or no broadband).

In [ ]:
# has_broadband
train_df['has_broadband'] = train_df['num_broadband'].apply(lambda x: 1 if x >= 1 else 0)
test_df['has_broadband'] = test_df['num_broadband'].apply(lambda x: 1 if x >= 1 else 0)

In [ ]:
# Separate labels from targets
train_x = train_df[['median_income', 'pct_broadband_customer_reported', 'pct_white', 'pct_black', 'pop_density', 'num_isp']]
train_y = train_df['has_broadband']

test_x = test_df[['median_income', 'pct_broadband_customer_reported', 'pct_white', 'pct_black', 'pop_density', 'num_isp']]
test_y = test_df['has_broadband']

Last class, we manually programmed a *grid search* to optimize model parameters. This time, we will use Scikit-Learn's built-in `GridSearchCV` class to do this automatically.

In [ ]:
# Cross-validation folds
k = 10

# Hyperparameters to tune:
params = {
    'criterion': {'gini', 'entropy'},
    'max_depth': {5, 10, 20},
    'min_samples_split': (2,5,10)
}

# Initialize GridSearchCV object with decision tree classifier and hyperparameters
grid_tree = GridSearchCV(estimator = DecisionTreeClassifier(random_state=0), param_grid=params, cv=k, return_train_score=True, scoring=['accuracy', 'precision', 'recall'], refit='accuracy')

# Train and cross-validate, print results
grid_tree.fit(train_x, train_y)
grid_tree_result = pd.DataFrame(grid_tree.dv.results_).sort_values(by=['mean_test_accuracy'], ascending=False)
grid_tree_result[['param_criterion', 'param_max_depth', 'param_min_samples_split', 'mean_test_accuracy']]


**DISCUSSION:** What are the optimal values of the hyperparameters? What might the `param_max_depth` values tell us about overfitting?

This means that we are able to predict broadband access with an **~79% cross-validation accuracy** (not test set accuracy) from the ACS demographic information. This may not seem great, but a high classification accuracy wasn't the primary goal of this exercise. Instead, we want to know  which features turn out ot be most predictive of broadband access to gain a better understanding of Chicago socioeconomics.

## Part 3. Interpreting Decision Trees

One way to interpret a decision tree is to just print the tree and see what features are used to divide up the data

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

# Display the tree with the highest test accuracy
best_tree = grid_tree.best_estimator_
plt.figure(figsize=(24,4), dpi=450)
tree.plot_tree(best_tree, filled = True, feature_names = train_x.columns.values, fontsize=3)
plt.show()

**DISCUSSION:** What does this tell us about broadband access?

We can also view the Gini feature importances that were automatically computed by the tree model when it was trained. This will tell us what was the most important feature that yielded the highest accuracy?

In [ ]:
# Create series of features sorted by importance
pd.Series(best_tree.feature_importances_, train_x.columns).sort_values(ascending=False)

**DISCUSSION:** This short list of numbers gives us a LOT to unpack. What are some things that this tells us about broadband access (as reported by ISPs)?

We can also plot a confusion matrix on the test set to see how the model is making mistakes. Remeber that the `1` class corresponds to "has broadband" and the `0` class corresponds to "No broadband"

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# Plot the confusion matrix


**DISCUSSION:** What does this tell us about broadband access?

Finally, we can print and plot the precision/recall for the test set

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, PrecisionRecallDisplay

# Test set predictions


# Print precision/recall/F1 on test set


# Display precision/recall curve
